1. Select columns justify choice
    - Maybe get some features from text data though embeddings (CLIP, BERT etc), n-grams etc
2. Breakdown into validation set
3. Standardization of data
4. Missingness analysis (sensitivity analysis maybe?)
5. Imputation of missing values
6. Distributions of the variables, unifariate analysis
    - Interesting observations about data etc. which hotels get higher reviews?
    - Topic modelling of property_summary
    - Use [kepler](https://kepler.gl/) to gain some geographic insights
    - Causal analysis
    - Hotels cluster analysis
    - Basket analysis - which ammenities frequently appear together?
7. Correlation analysis/Multicollinearity
8. Outlier analysis?
9. Variable importance, shapley values
10. Model feature/selection
    - Maybe PCA?
11. Regularization?
12. Model comparison based on metrics, validation
13. Hyperparameter tuning/ cross validation
14. Choice of loss functions

In [78]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [79]:
df = pd.read_csv("./train.csv")

In [80]:
def get_num_items(item):
    if not pd.isna(item):
        return len(item.split(","))
    else:
        item

In [81]:
df.drop(columns="property_id").drop_duplicates().shape[0]

6495

In [82]:
df['num_amenities'] = df['property_amenities'].apply(get_num_items)
df['n_verified_channels'] = df['host_verified'].apply(get_num_items)

In [83]:
features = ['property_type', 
            'property_room_type', 
            'property_max_guests',
            'property_bathrooms',
            'property_bedrooms',
            'property_beds',
            'property_bed_type',
            'num_amenities',
            'host_since',
            'host_location',
            'host_response_time',
            'host_response_rate',
            'n_verified_channels',
            'booking_price_covers',
            'booking_min_nights',
            'booking_max_nights',
            'booking_availability_30',
            'booking_availability_60',
            'booking_availability_90',
            'booking_availability_365',
            'booking_cancel_policy',
            'reviews_num',
            'reviews_rating',
            'reviews_acc',
            'reviews_cleanliness', 
            'reviews_checkin',
            'reviews_communication',
            'reviews_location', 
            'reviews_value', 
            'reviews_per_month',
            'target']

In [84]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

newdf = df[features].select_dtypes(include=numerics)

In [86]:
newdf.host_response_rate

0       100.0
1        75.0
2       100.0
3       100.0
4       100.0
        ...  
6490    100.0
6491    100.0
6492      NaN
6493      NaN
6494      NaN
Name: host_response_rate, Length: 6495, dtype: float64

newdf.isnull().sum()

In [71]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(newdf)
X_scaled = scaler.transform(newdf)

In [72]:
scaled_df = pd.DataFrame(X_scaled)
scaled_df.columns = newdf.columns

In [73]:
corrr = scaled_df.corr()


In [39]:
import matplotlib

In [76]:
scaled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6495 entries, 0 to 6494
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   property_max_guests       6495 non-null   float64
 1   property_bathrooms        6483 non-null   float64
 2   property_bedrooms         6486 non-null   float64
 3   property_beds             6484 non-null   float64
 4   num_amenities             6443 non-null   float64
 5   host_response_rate        5034 non-null   float64
 6   n_verified_channels       6495 non-null   float64
 7   booking_price_covers      6495 non-null   float64
 8   booking_min_nights        6495 non-null   float64
 9   booking_max_nights        6495 non-null   float64
 10  booking_availability_30   6495 non-null   float64
 11  booking_availability_60   6495 non-null   float64
 12  booking_availability_90   6495 non-null   float64
 13  booking_availability_365  6495 non-null   float64
 14  reviews_

In [40]:
corrr.style.background_gradient(cmap='coolwarm')

,property_max_guests,property_bathrooms,property_bedrooms,property_beds,num_amenities,host_response_rate,n_verified_channels,booking_price_covers,booking_min_nights,booking_max_nights,booking_availability_30,booking_availability_60,booking_availability_90,booking_availability_365,reviews_num,reviews_rating,reviews_acc,reviews_cleanliness,reviews_checkin,reviews_communication,reviews_location,reviews_value,reviews_per_month,target
property_max_guests,1.000000,0.350172,0.635758,0.817748,0.228699,0.061773,0.057782,0.525823,-0.023394,0.022501,0.037073,0.036343,0.030952,0.080033,0.057177,-0.036613,-0.044361,-0.040600,-0.027932,-0.023900,-0.015755,-0.054618,-0.007558,0.013431
property_bathrooms,0.350172,1.000000,0.422434,0.333005,0.112302,-0.024011,0.015054,0.211820,-0.013256,-0.007676,0.020518,0.006287,0.001721,0.008936,-0.040935,0.028332,-0.003639,-0.000869,-0.019082,-0.013873,-0.008874,0.009846,-0.048229,-0.009437
property_bedrooms,0.635758,0.422434,1.000000,0.610399,0.163357,-0.016055,0.040877,0.368220,0.000978,0.019590,-0.001942,-0.016183,-0.020938,-0.010192,-0.050951,0.034052,0.015346,0.000941,0.020904,0.021501,-0.026108,0.019635,-0.080886,0.006142
property_beds,0.817748,0.333005,0.610399,1.000000,0.185319,0.021501,0.017991,0.441048,-0.017739,0.039508,0.045879,0.043440,0.039213,0.075824,0.046735,-0.035972,-0.060783,-0.029234,-0.020347,-0.019041,-0.030981,-0.046543,-0.019697,0.009198
num_amenities,0.228699,0.112302,0.163357,0.185319,1.000000,0.140709,0.105252,0.166478,-0.010311,-0.013137,0.092952,0.115685,0.120426,0.146858,0.107478,0.153731,0.136395,0.171101,0.097623,0.096570,0.097788,0.115410,0.142849,-0.012557
host_response_rate,0.061773,-0.024011,-0.016055,0.021501,0.140709,1.000000,0.116141,0.035582,0.012945,0.005809,-0.172297,-0.155004,-0.156057,-0.100113,0.127008,0.067458,0.072497,0.123841,0.041266,0.051790,0.033435,0.046958,0.178247,0.004598
n_verified_channels,0.057782,0.015054,0.040877,0.017991,0.105252,0.116141,1.000000,0.038925,0.000699,-0.000732,-0.002431,0.017488,0.029195,0.049124,0.074264,0.072769,0.060826,0.043174,0.012272,0.045259,0.042999,0.042118,0.077808,-0.001761
booking_price_covers,0.525823,0.211820,0.368220,0.441048,0.166478,0.035582,0.038925,1.000000,-0.017056,0.036342,0.030272,0.033760,0.034626,0.094798,0.081769,-0.024482,-0.018992,-0.011816,-0.019636,-0.026159,-0.009587,-0.034171,-0.000207,0.007298
booking_min_nights,-0.023394,-0.013256,0.000978,-0.017739,-0.010311,0.012945,0.000699,-0.017056,1.000000,-0.002517,-0.014277,-0.020149,-0.018130,0.001203,-0.038771,-0.001980,-0.005318,0.006038,0.006972,0.005514,0.002050,-0.016704,-0.090918,0.007518
booking_max_nights,0.022501,-0.007676,0.019590,0.039508,-0.013137,0.005809,-0.000732,0.036342,-0.002517,1.000000,0.032001,0.029997,0.028729,0.033063,0.007362,-0.039248,-0.041713,-0.019774,-0.013093,-0.013653,-0.024031,-0.031797,-0.011108,-0.015577


In [55]:
imp = IterativeImputer(max_iter=10, random_state=0)

In [57]:
fitted = imp.fit(X_scaled)

/Users/ehambardzumy/Documents/big_data/lib/python3.9/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [60]:
X_scaled = fitted.transform(X_scaled)

In [62]:
scaled_df = pd.DataFrame(X_scaled)
scaled_df.columns = newdf.columns

In [63]:
scaled_df.corr().style.background_gradient(cmap='coolwarm')

,property_max_guests,property_bathrooms,property_bedrooms,property_beds,num_amenities,host_response_rate,n_verified_channels,booking_price_covers,booking_min_nights,booking_max_nights,booking_availability_30,booking_availability_60,booking_availability_90,booking_availability_365,reviews_num,reviews_rating,reviews_acc,reviews_cleanliness,reviews_checkin,reviews_communication,reviews_location,reviews_value,reviews_per_month,target
property_max_guests,1.000000,0.350058,0.635840,0.817831,0.229597,0.068562,0.057782,0.525823,-0.023394,0.022501,0.037073,0.036343,0.030952,0.080033,0.057177,-0.033883,-0.041339,-0.035917,-0.025883,-0.019438,-0.013356,-0.051195,-0.000239,0.013431
property_bathrooms,0.350058,1.000000,0.422630,0.333223,0.113074,-0.028696,0.014786,0.211874,-0.013177,-0.007669,0.020812,0.006576,0.002014,0.009232,-0.041280,0.036328,-0.001672,-0.003272,-0.025996,-0.018624,-0.009958,0.013907,-0.051560,-0.009603
property_bedrooms,0.635840,0.422630,1.000000,0.610131,0.163376,-0.014119,0.041041,0.368340,0.000996,0.019601,-0.001795,-0.016020,-0.020848,-0.010369,-0.051131,0.044851,0.022980,0.004486,0.027108,0.029692,-0.026163,0.028727,-0.082106,0.006241
property_beds,0.817831,0.333223,0.610131,1.000000,0.185902,0.025213,0.018685,0.441267,-0.017684,0.039522,0.044973,0.042642,0.038514,0.075385,0.046828,-0.034966,-0.062634,-0.027781,-0.017980,-0.015577,-0.031775,-0.044252,-0.018931,0.008595
num_amenities,0.229597,0.113074,0.163376,0.185902,1.000000,0.155850,0.107908,0.167385,-0.010116,-0.013062,0.090957,0.114017,0.119021,0.145830,0.108517,0.169337,0.148924,0.189572,0.104974,0.103281,0.105715,0.122875,0.159329,-0.012566
host_response_rate,0.068562,-0.028696,-0.014119,0.025213,0.155850,1.000000,0.131954,0.039643,0.016599,0.006185,-0.151686,-0.130195,-0.124600,-0.076168,0.130812,0.106382,0.114414,0.192293,0.064218,0.082057,0.055376,0.078037,0.212891,0.002010
n_verified_channels,0.057782,0.014786,0.041041,0.018685,0.107908,0.131954,1.000000,0.038925,0.000699,-0.000732,-0.002431,0.017488,0.029195,0.049124,0.074264,0.090054,0.079910,0.071615,0.016949,0.056668,0.053605,0.052909,0.120598,-0.001761
booking_price_covers,0.525823,0.211874,0.368340,0.441267,0.167385,0.039643,0.038925,1.000000,-0.017056,0.036342,0.030272,0.033760,0.034626,0.094798,0.081769,-0.020710,-0.013349,-0.002087,-0.016746,-0.023269,-0.005847,-0.030819,0.017293,0.007298
booking_min_nights,-0.023394,-0.013177,0.000996,-0.017684,-0.010116,0.016599,0.000699,-0.017056,1.000000,-0.002517,-0.014277,-0.020149,-0.018130,0.001203,-0.038771,-0.006605,-0.012086,0.010453,0.011536,0.008450,0.002205,-0.033899,-0.119193,0.007518
booking_max_nights,0.022501,-0.007669,0.019601,0.039522,-0.013062,0.006185,-0.000732,0.036342,-0.002517,1.000000,0.032001,0.029997,0.028729,0.033063,0.007362,-0.038648,-0.040805,-0.018394,-0.012697,-0.013238,-0.023581,-0.031001,-0.008355,-0.015577
